In [1]:
#!/usr/bin/env python3
"""
Genera:
  • summary_scenarios.csv  –  métrica global por escenario
  • mt_distribution.csv    –  |mt| de cada par descoordinado
Notas:
  – TMT se define como la suma de mt (siempre negativo) y es adimensional.
  – CTI fijo en 0.2 s.
"""
import json, csv, os
CTI = 0.2
INPUT_FILE = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json"
OUT_CSV_SCEN = "summary_scenarios_optimized.csv"
OUT_CSV_MT   = "mt_distribution_optimized.csv"

with open(INPUT_FILE) as f:
    data = json.load(f)

per_scenario = {}
for p in data:
    sid = p["scenario_id"]
    d = per_scenario.setdefault(sid, {"total":0,"valid":0,"descoord":0,"TMT":0.0})
    d["total"] += 1
    tm, tb = p["main_relay"].get("Time_out"), p["backup_relay"].get("Time_out")
    if not all(isinstance(x,(int,float)) for x in (tm, tb)):
        continue
    d["valid"] += 1
    delta_t = tb - tm - CTI
    mt = (delta_t - abs(delta_t))/2        # negativo si Δt<0
    if delta_t < 0:
        d["descoord"] += 1
        d["TMT"]      += mt                # suma valores negativos
        # exportar |mt| para gráfica de distribución
        with open(OUT_CSV_MT,"a",newline="") as g:
            csv.writer(g).writerow([
                sid,
                f'{p["fault"]}% {p["main_relay"]["relay"]}/{p["backup_relay"]["relay"]}',
                abs(mt)
            ])

with open(OUT_CSV_SCEN,"w",newline="") as f:
    w = csv.writer(f)
    w.writerow(["Escenario","Pares evaluados","Pares válidos",
                "Descoordinados","% Descoord","TMT (negativo)"])
    for sid,d in sorted(per_scenario.items(),
                        key=lambda k:int(k[0].split("_")[-1])):
        pct = 100*d["descoord"]/d["valid"] if d["valid"] else 0
        w.writerow([sid,d["total"],d["valid"],d["descoord"],
                    f"{pct:.1f}",f"{d['TMT']:.5f}"])
print("✓ summary_scenarios.csv y mt_distribution.csv listos")


✓ summary_scenarios.csv y mt_distribution.csv listos


# graph


In [2]:
#!/usr/bin/env python3
"""
Lee summary_scenarios.csv y genera:
  • tmt_by_scenario.png  –  gráfica en barras del TMT (negativo)
"""
import pandas as pd
import matplotlib.pyplot as plt

# --- 1. Cargar CSV ---
df = pd.read_csv("summary_scenarios_optimized.csv")

# --- 2. Extraer número de escenario y ordenar ---
df["num"] = pd.to_numeric(df["Escenario"].str.extract(r"(\d+)")[0])
df = df.sort_values("num")

# --- 3. Graficar ---
plt.figure(figsize=(12,5))
plt.bar(df["Escenario"], df["TMT (negativo)"])
plt.title("TMT por escenario (68 escenarios)")
plt.xlabel("Escenario")
plt.ylabel("TMT")
plt.xticks(rotation=90)
plt.tight_layout()

# --- 4. Guardar como PNG (300 dpi) ---
plt.savefig("tmt_by_scenario_optimized.png", dpi=300)
plt.close()

print("✓ tmt_by_scenario.png generado")


✓ tmt_by_scenario.png generado
